In [13]:
from typing import List
import copy

class Utility:

    # A set of the basic moves a piece can make.
    moves = {
        "k" : [[0,1], [1, 0], [0, -1], [-1, 0], [1, 1], [1, -1], [-1, 1], [-1, -1], [-2, 0], [2, 0]],
        "n" : [[1, 2], [1, -2], [-1, 2], [-1, -2,], [2, 1], [2, -1], [-2, 1], [-2, -1]],
        "p" : [[0, 1], [0, 2], [1, 1], [-1, 1]],
        "r" : [
            [0,1], [1, 0], [0, -1], [-1, 0], 
            [0,2], [2, 0], [0, -2], [-2, 0], 
            [0,3], [3, 0], [0, -3], [-3, 0], 
            [0,4], [4, 0], [0, -4], [-4, 0], 
            [0,5], [5, 0], [0, -5], [-5, 0], 
            [0,6], [6, 0], [0, -6], [-6, 0], 
            [0,7], [7, 0], [0, -7], [-7, 0]],
        "b" : [
            [1, 1], [1, -1], [-1, 1], [-1, -1],
            [2, 2], [2, -2], [-2, 2], [-2, -2],
            [3, 3], [3, -2], [-3, 3], [-3, -3],
            [4, 4], [4, -4], [-4, 4], [-4, -4],
            [5, 5], [5, -5], [-5, 5], [-5, -5],
            [6, 6], [6, -6], [-6, 6], [-6, -6],
            [7, 7], [7, -7], [-7, 7], [-7, -7]],
    }

    # The queen can make the moves a Bishop and Rook can make.
    moves["q"] = moves["b"] + moves["r"]

    # The values of pieces on the white team, negated if team is black.
    values = {
        "k" : 999999,
        "q" : 9,
        "r" : 5,
        "b" : 3,
        "n" : 3,
        "p" : 1
    }

    # A list of the unicodes for the white team.
    whiteIcons = {
        "k" : "\u265A",
        "q" : "\u265B",
        "r" : "\u265C",
        "b" : "\u265D",
        "n" : "\u265E",
        "p" : "\u265F"
    }

    # A list of unicodes for the black team.
    blackIcons = {
        "k" : "\u2654",
        "q" : "\u2655",
        "r" : "\u2656",
        "b" : "\u2657",
        "n" : "\u2658",
        "p" : "\u2659"
    }

    # A list of position evaluations for each piece on the white team. Inverse if team is black.
    positionValues = {
        "k": [
        [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
        [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
        [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
        [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
        [-2.0, -3.0, -3.0, -4.0, -4.0, -3.0, -3.0, -2.0],
        [-1.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -1.0],
        [2.0,  2.0,  0.0,  0.0,  0.0,  0.0,  2.0,  2.0],
        [2.0,  3.0,  1.0,  0.0,  0.0,  1.0,  3.0,  2.0]],

        "q": [
        [-2.0, -1.0, -1.0, -0.5, -0.5, -1.0, -1.0, -2.0],
        [-1.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, -1.0],
        [-1.0,  0.0,  0.5,  0.5,  0.5,  0.5,  0.0, -1.0],
        [-0.5,  0.0,  0.5,  0.5,  0.5,  0.5,  0.0, -0.5],
        [0.0,  0.0,  0.5,  0.5,  0.5,  0.5,  0.0, -0.5],
        [-1.0,  0.5,  0.5,  0.5,  0.5,  0.5,  0.0, -1.0],
        [-1.0,  0.0,  0.5,  0.0,  0.0,  0.0,  0.0, -1.0],
        [-2.0, -1.0, -1.0, -0.5, -0.5, -1.0, -1.0, -2.0]],

         "r": [
        [0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0],
        [0.5,  1.0,  1.0,  1.0,  1.0,  1.0,  1.0,  0.5],
        [-0.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, -0.5],
        [-0.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, -0.5],
        [-0.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, -0.5],
        [-0.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, -0.5],
        [-0.5,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, -0.5],
        [0.0,   0.0, 0.0,  0.5,  0.5,  0.0,  0.0,  0.0]],

        "b": [
        [-2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -2.0],
        [-1.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, -1.0],
        [-1.0,  0.0,  0.5,  1.0,  1.0,  0.5,  0.0, -1.0],
        [-1.0,  0.5,  0.5,  1.0,  1.0,  0.5,  0.5, -1.0],
        [-1.0,  0.0,  1.0,  1.0,  1.0,  1.0,  0.0, -1.0],
        [-1.0,  1.0,  1.0,  1.0,  1.0,  1.0,  1.0, -1.0],
        [-1.0,  0.5,  0.0,  0.0,  0.0,  0.0,  0.5, -1.0],
        [-2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -2.0]],
       
        "n": [
        [-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -4.0, -5.0],
        [-4.0, -2.0,  0.0,  0.0,  0.0,  0.0, -2.0, -4.0],
        [-3.0,  0.0,  1.0,  1.5,  1.5,  1.0,  0.0, -3.0],
        [-3.0,  0.5,  1.5,  2.0,  2.0,  1.5,  0.5, -3.0],
        [-3.0,  0.0,  1.5,  2.0,  2.0,  1.5,  0.0, -3.0],
        [-3.0,  0.5,  1.0,  1.5,  1.5,  1.0,  0.5, -3.0],
        [-4.0, -2.0,  0.0,  0.5,  0.5,  0.0, -2.0, -4.0],
        [-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -4.0, -5.0]],

        "p": [
        [0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0],
        [5.0,  5.0,  5.0,  5.0,  5.0,  5.0,  5.0,  5.0],
        [1.0,  1.0,  2.0,  3.0,  3.0,  2.0,  1.0,  1.0],
        [0.5,  0.5,  1.0,  2.5,  2.5,  1.0,  0.5,  0.5],
        [0.0,  0.0,  0.0,  2.0,  2.0,  0.0,  0.0,  0.0],
        [0.5, -0.5, -1.0,  0.0,  0.0, -1.0, -0.5,  0.5],
        [0.5,  1.0, 1.0,  -2.0, -2.0,  1.0,  1.0,  0.5],
        [0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0]],
        
    }

    @staticmethod
    def convertToNumber(letter: str) -> int:
        return ord(letter) - 97

    @staticmethod
    def convertToLetter(number: int) -> chr:
        return chr(number + 97)
    
    @staticmethod
    def convertStringMoveToInt(move: str) -> List[int]:
        moves = []

        for char in range(len(move)):
            if char % 2 == 0:
                moves.append(Utility.convertToNumber(move[char]))
            else:
                moves.append(int(move[char]) - 1)

        return moves

    @staticmethod
    def convertIntMoveToString(move: List[int]) -> str:
        moves = ""

        for char in range(len(move)):
            if char % 2 == 0:
                moves += Utility.convertToLetter(move[char])
            else:
                moves += str(move[char] + 1)

        return moves

    @staticmethod
    def negateArray(array: List[List[int]]) -> List[List[int]]:

        negativeArray = copy.deepcopy(array)

        for row in range(len(array)):
            for column in range(len(array[row])):
                negativeArray[row][column] *= -1

        return negativeArray

    @staticmethod
    def inverseArray(array: List[List[int]]) -> List[List[int]]:
        return list(reversed(array))

    @staticmethod
    def getBasicMoves(team: chr, name: chr) -> List[List[int]]:

        if team == "W":
            return Utility.moves[name]
        else:
            return Utility.negateArray(Utility.moves[name])

    @staticmethod
    def getPieceEvaluation(team: chr, name: chr) -> int:

        if team == "W":
            return Utility.values[name]
        else:
            return -Utility.values[name]

    @staticmethod
    def getPositionEvaluation(team: chr, name: chr, x: int, y: int) -> int:

        if team == "W":
            return (Utility.positionValues[name])[y][x]
        else:
            return (Utility.negateArray(Utility.inverseArray(Utility.positionValues[name])))[y][x]

    @staticmethod
    def getPieceIcon(team: chr, name: chr) -> str:

        if team == "W":
            return Utility.whiteIcons[name]
        else:
            return Utility.blackIcons[name]
class Piece():
    
    def __init__(self, name: chr, team: chr, board, x: int, y: int):
        self.name = name
        self.team = team
        self.board = board
        self.x = x
        self.y = y
        self.value = Utility.getPieceEvaluation(team, name)
        self.icon = Utility.getPieceIcon(team, name) 
        self.basicMoves = Utility.getBasicMoves(team, name) 
        self.played = False 
        self.possibleMoves = [] 

    def updatePossibleMoves(self):

        self.possibleMoves = [] 
        
        # Test if each basic move is a possible move.
        for move in self.basicMoves:

            currentPosition = [self.x, self.y]
            UpdatedMove = [move[0] + self.x, move[1] + self.y]
            if self.board.contains(UpdatedMove[0], UpdatedMove[1]) == False:
                continue
            newMove = currentPosition + UpdatedMove
            stringMove = Utility.convertIntMoveToString(newMove)

            if self.isValidMove(stringMove) and move not in self.possibleMoves:
                #print("moveeeeeeeeeeeeeeeee: ",stringMove)
                self.possibleMoves.append(stringMove)
            
    def isValidMove(self, move: str) -> bool:
        
        intMove = Utility.convertStringMoveToInt(move)
        newX = intMove[2]
        newY = intMove[3]

        diffX = newX - self.x
        diffY = newY - self.y

        if [diffX, diffY] in self.basicMoves:

            if self.isValidPath(move):
                return True

        return False

    def isValidPath(self, move: str) -> bool:

        intMove = Utility.convertStringMoveToInt(move)
        newX = intMove[2]
        newY = intMove[3]
        newMove = [newX, newY]

        diffX = newX - self.x
        diffY = newY - self.y

        
        if diffX != 0:
            stepX = int(diffX / abs(diffX)) 
        else:
            stepX = 0
        
        if diffY != 0:
            stepY = int(diffY / abs(diffY))
        else:
            stepY = 0

        tempX = self.x + stepX
        tempY = self.y + stepY

        tempMove = [tempX, tempY]

        if self.board.contains(newX, newY) == False:
            return False

        if self.board.isEmpty(newMove[0], newMove[1]) == False and self.board.isFriendly(newMove[0], newMove[1], self.team) == True:
            return False
        else:
            while tempMove != newMove:

                if self.board.contains(tempMove[0], tempMove[1]) == False:
                    return False
                elif self.board.isEmpty(tempMove[0], tempMove[1]) == False:
                    return False
                else:
                    tempMove[0] += stepX
                    tempMove[1] += stepY

        return True

    def counterUpdate(self, x: int, y: int):

        if self.board.canAttack(x, y, self.team)==True:
            self.board.counterReset() # Resets counter a piece is going to be removed.
        else:
            self.board.counterIncrement() # Increment counter otherwise.

    def move(self, move: str) -> bool:

        if move in self.possibleMoves:
            intMove = Utility.convertStringMoveToInt(move)
            currentX = intMove[0]
            currentY = intMove[1]
            newX = intMove[2]
            newY = intMove[3]
            self.board.removePiece(currentX, currentY)
            self.counterUpdate(newX, newY)
            self.board.addPiece(self, newX, newY)
            self.x = newX
            self.y = newY

            if self.played == False:
                self.played = True
            
            self.board.lastMove = self.name + move
        else:
            print(move + "  is an illegal move. Try again")
            return False

    def __repr__(self):
        return self.icon
    
class Queen(Piece):

    def __init__(self, team, board, x, y):
        super().__init__("q", team, board, x, y)

class Rook(Piece):

    def __init__(self, team, board, x, y):
        super().__init__("r", team, board, x, y)
class Bishop(Piece):

    def __init__(self, team, board, x, y):
        super().__init__("b", team, board, x, y)        

class Knight(Piece):

    def __init__(self, team, board, x, y):
        super().__init__("n", team, board, x, y)

    def isValidMove(self, move: str) -> bool:

        intMove = Utility.convertStringMoveToInt(move)
        newX = intMove[2]
        newY = intMove[3]

        diffX = newX - self.x
        diffY = newY - self.y

        if [diffX, diffY] in self.basicMoves:

            if self.board.isEmpty(newX, newY) == True or self.board.canAttack(newX, newY, self.team):
                return True

        return False

class King(Piece):

    def __init__(self, team, board, x, y):
        super().__init__("k", team, board, x, y)
        self.castleAttempt = False
    
    def isValidMove(self, move: str) -> bool:

        intMove = Utility.convertStringMoveToInt(move)
        newX = intMove[2]
        newY = intMove[3]

        diffX = newX - self.x
        diffY = newY - self.y

        if [diffX, diffY] in self.basicMoves:

            # Check if a Castle can be made.
            if self.played == False and abs(diffX) == 2:
                if diffX == 2:
                    if self.board.isEmpty(7, self.y) == False and type(self.board.board[self.y][7]) is Rook:
                        moveAttempt = Utility.convertIntMoveToString([self.x, self.y, 6, self.y])
                        if self.board.board[self.y][7].played == False and self.isValidPath(moveAttempt):
                            self.castleAttempt = True
                            return True
                elif diffX == -2:
                    if self.board.isEmpty(0, self.y) == False and type(self.board.board[self.y][0]) is Rook:
                        moveAttempt = Utility.convertIntMoveToString([self.x, self.y, 1, self.y])
                        if self.board.board[self.y][0].played == False and self.isValidPath(moveAttempt):
                            self.castleAttempt = True
                            return True
                else:
                    return False
            elif self.isValidPath(move):
                return True
            elif self.board.isEmpty(newX, newY) == True or self.board.canAttack(newX, newY, self.team):
                return True
            else:
                return False

        return False
#..........................MOVE.....................
    def move(self, move: str) -> bool:

        if move in self.possibleMoves:
            intMove = Utility.convertStringMoveToInt(move)
            currentX = intMove[0]
            currentY = intMove[1]
            newX = intMove[2]
            newY = intMove[3]
            self.board.removePiece(currentX, currentY)
            self.counterUpdate(newX, newY)
            self.board.addPiece(self, newX, newY)
            self.x = newX
            self.y = newY

            if self.castleAttempt == True:

                if newX > 4:
                    self.board.board[self.y][5] = self.board.board[self.y][7]
                    self.board.removePiece(7, self.y)
                if newX < 4:
                    self.board.board[self.y][3] = self.board.board[self.y][0]
                    self.board.removePiece(0, self.y)
                
                self.castleAttempt = False

            if self.played == False:
                self.played = True
            
            self.board.lastMove = self.name + move
        else:
            print(move + " " + "is an illegal move. Try again")
            return False
class Pawn(Piece):

    def __init__(self, team, board, x, y):
        super().__init__("p", team, board, x, y)
        self.passentAttempt = False
        self.isAI = False
    
    def isValidMove(self, move: str) -> bool:

        intMove = Utility.convertStringMoveToInt(move)
        newX = intMove[2]
        newY = intMove[3]

        diffX = newX - self.x
        diffY = newY - self.y

        if [diffX, diffY] in self.basicMoves:

            if abs(diffX) == 1 and abs(diffY) == 1:

                if self.board.canAttack(newX, newY, self.team):
                    return True
                else:
                    lastMove = self.board.lastMove

                    if lastMove != "" and lastMove[0] == "p":
                        intMoves = Utility.convertStringMoveToInt(lastMove[1:5])
                        currentX = intMoves[0]
                        currentY = intMoves[1]
                        newX = intMoves[2]
                        newY = intMoves[3]

                        if abs(newY - self.y) != 1:
                            return False

                        if newX - currentX == 0 and abs(newY - currentY) == 2:
                            if currentX + 1 == self.x or currentX - 1 == self.x:
                                self.passentAttempt = True
                                return True
                    else:
                        return False
                     
            elif abs(diffX) == 0 and abs(diffY) == 2:

                if self.played == False and self.board.isEmpty(newX, newY):
                    if self.isValidPath(move):
                        return True
                    else:
                        return False
                else:
                    return False

            else:
                if self.board.isEmpty(newX, newY):
                    return True
        else:
            return False

        return False

    
#Moves the piece to a new position and promotes the piece if requirements are met
    
    def move(self, move: str) -> bool:

        if move in self.possibleMoves:
            intMove = Utility.convertStringMoveToInt(move)
            currentX = intMove[0]
            currentY = intMove[1]
            newX = intMove[2]
            newY = intMove[3]
            self.board.removePiece(currentX, currentY)
            self.board.counterReset()
            if self.passentAttempt == True:
                if self.team == "W":
                    self.board.removePiece(newX, newY - 1)
                else:
                    self.board.removePiece(newX, newY + 1)
                self.passentAttempt == False
            self.board.addPiece(self, newX, newY)

            self.x = newX
            self.y = newY

            if self.played == False:
                self.played = True

            if newY == 0 or newY == 7:

                if self.isAI == False:
                    name = input("What piece would like to upgrade to. Enter the first lower case letter of the piece, n for knight:")
                    choices = ["q", "n", "b", "r"]
                    if name not in choices :
                        while name not in choices:
                            name = input("What piece would like to upgrade to. Enter the first lower case letter of the piece, n for knight:")

                            if name in choices:
                                break
                else:
                    name = "q"

                self.board.promotePawn(self.x, self.y, self.team, name)
            
            self.board.lastMove = self.name + move
        else:
            print(move + " " + "is an illegal move. Try again")
            return False
class Board():

   #creates a board
    def __init__(self):
        self.board = [[None for x in range(8)] for y in range(8)] # A 8x8 grid holding all the pieces in the game.
        self.lastMove = "" # The last move made in the game.
        self.counter = 0 # The number of moves since the last pawn move or capture.

    
    #function Checks if an empty spot exists at (x,y).
    def isEmpty(self, x, y) -> bool:
        return self.board[y][x] == None


    # Check if a friendly piece exists at (x,y).
    def isFriendly(self, x, y, team) -> bool:
        return self.board[y][x].team == team


    #Checks if a piece can be attacked at (x,y).
    def canAttack(self, x, y, team) -> bool:
        return self.isEmpty(x, y) == False and self.isFriendly(x, y, team) == False

    #Checks if a point exists on the board.
    def contains(self, x, y) -> bool:
        return x >= 0 and x < 8 and y >= 0 and y < 8

    def getAllPossibleMoves(self, team) -> List[str]:

        possibleMoves = []

        for row in self.board:
            for piece in row:
               
                if piece != None and piece.team == team:
                     #print("piece teammmmmmms: ",piece.team,"\n")
                    piece.updatePossibleMoves()
                    possibleMoves += piece.possibleMoves

        return possibleMoves

   
    def counterReset(self):
        self.counter = 0

    def counterIncrement(self):
        self.counter += 1

   #Places all the pieces on the board.
    def fillBoard(self):
        startingPositions = [1, 6]
        teams = ["W", "B"]

        for index in range(2):
            for x in range(8):
                self.board[startingPositions[index]][x] = Pawn(
                    teams[index], self, x, startingPositions[index])

        startingPositions = [0, 7]

        for index in range(2):
            self.board[startingPositions[index]][0] = Rook(
                teams[index], self, 0, startingPositions[index])
            self.board[startingPositions[index]][1] = Knight(
                teams[index], self, 1, startingPositions[index])
            self.board[startingPositions[index]][2] = Bishop(
                teams[index], self, 2, startingPositions[index])
            self.board[startingPositions[index]][3] = Queen(
                teams[index], self, 3, startingPositions[index])
            self.board[startingPositions[index]][4] = King(
                teams[index], self, 4, startingPositions[index])
            self.board[startingPositions[index]][5] = Bishop(
                teams[index], self, 5, startingPositions[index])
            self.board[startingPositions[index]][6] = Knight(
                teams[index], self, 6, startingPositions[index])
            self.board[startingPositions[index]][7] = Rook(
                teams[index], self, 7, startingPositions[index])

 #print current board
    def printBoard(self):

        print("  " + "A  B  C  D  E  F  G  H" + "\n")
        line = ""

        for y in range(8):

            line = line + str(y + 1)

            for x in range(8):
                if self.board[y][x] == None:
                    line = line + " " + "-"
                else:
                    line = line + " " + repr(self.board[y][x])

            print(line + "  " + str(y + 1) + "\n")
            line = ""

        print("  " + "A  B  C  D  E  F  G  H" + "\n")


    #Declares all pawns on the specified team as an A.I.
    def declareAsAI(self, team: chr):

        for row in self.board:
            for piece in row:
                if piece != None and piece.name == "p" and piece.team == team:
                    piece.isAI = True
    

    #Make a move on the board.       
    def makeMove(self, move: str) -> bool:
        
        intMoves = Utility.convertStringMoveToInt(move)
        xToCall = intMoves[0]
        yToCall = intMoves[1]

        if self.board[yToCall][xToCall] != None:
            self.board[yToCall][xToCall].updatePossibleMoves()
        else:
            print("{} is a illegal move. No piece exists at ({},{}) Try again.".format(move, xToCall, yToCall))
            return False

        if move in self.board[yToCall][xToCall].possibleMoves:
            self.board[yToCall][xToCall].move(move)
            return True
        else:
            print("{} is a illegal move. It's not a possible move. Try again.".format(move))
            return False

    #Add a piece on the board.        
    def addPiece(self, piece: Piece, x: int, y: int):
        self.board[y][x] = piece

    #Remove a piece on the board.
    def removePiece(self, x: int, y: int):
        self.board[y][x] = None

    #Gets an evaluation of the board.
    def getEvaluation(self) -> int:

        evaluation = 0

        for row in self.board:
            for piece in row:
                if piece != None:
                    evaluation += (piece.value + Utility.getPositionEvaluation(piece.team, piece.name, piece.x, piece.y))
        
        return evaluation

    #Determines if the 50-move rule has occurred.
    def canDeclareDraw(self) -> bool:

        if self.counter >= 50:
            return True
        else:
            return False
    

    #Gets the king from the specified team.
    def getKing(self, team: chr):
        
        for row in self.board:
            for piece in row:
                if piece != None and type(piece) == King and piece.team == team:
                    return piece
        
        return None

   
    # Determines if a team is in checkmate.
    def isCheckmate(self, team: chr) -> bool:
        
        for move in self.getAllPossibleMoves(team):
            boardCopy = self.getDeepCopy()
            boardCopy.makeMove(move)

            if boardCopy.isCheck(team) == False:
                return False

        return True
    
    # Determines if a team is in check.
    def isCheck(self, team: chr) -> bool:

        king = self.getKing(team)
        kingX = Utility.convertToLetter(king.x)
        kingY = king.y
        
        if team == "W":
            possibleMoves = self.getAllPossibleMoves("B")
        else:
            possibleMoves = self.getAllPossibleMoves("W")

        for move in possibleMoves:
            
            if move[2] == kingX and int(move[3]) - 1 == kingY:
                return True

        return False

    #desc: Determines a stalemate has occurred.
    def isStalemate(self, team: chr) -> bool:
        
        if self.isCheckmate(team) == False:

            for move in self.getAllPossibleMoves(team):
                boardCopy = self.getDeepCopy()
                boardCopy.makeMove(move)

                if boardCopy.isCheck(team) == False:
                    return False
            
            return True

        return False
    
    def getDeepCopy(self):
        return copy.deepcopy(self)


    #Promotes a pawn to a different piece at the specified location.
    def promotePawn(self, x: int, y: int, team: chr, name: chr):

        if name == "q":
            piece = Queen(team, self, x, y)
        elif name == "b":
            piece = Bishop(team, self, x, y)
        elif name == "n":
            piece = Knight(team, self, x, y)
        else:
            piece = Rook(team, self, x, y)
        
        self.removePiece(x, y)
        self.addPiece(piece, x, y)
        
class Player():

    def __init__(self, team: chr, board, isAI):
        self.team = team
        self.board = board
        self.isAI = isAI

    #Manipulates pieces on the game board.
    def move(self, move: str):

        self.board.makeMove(move)

    def getAllPossibleMoves(self):
        return self.board.getAllPossibleMoves(self.team)

    #Helper method to assist in the retrieval of the best move.
    def getBestPossibleMove(self, depth):
        if self.team == "W":
            team = True
        else:
            team = False

        return self.minimax(depth, team)

    # Gets the best possible move a player can make by evaluating future logical moves and basing its own evaluation off that.

    def minimax(self, depth, PlayerWhite) -> str:

        # Check if player is attempting to maximize or minimize the board evaluation.
        if PlayerWhite==True:
            bestPossibleMove = -999999 # Start with some arbitrary low number.
            team = "W" # Declare that the current team is white.
        else:
            bestPossibleMove = 999999 # Start with some arbitrary high number.
            team = "B" # Declare that the current team is black.

        # All the possible moves that can be made.
        moves = self.board.getAllPossibleMoves(team)
        bestPossibleMoveFound = "" # Holds the best possible move that can made.

        # Iterate through each possible move to find which optimizes the reward.
        for move in moves:
            boardCopy = self.board.getDeepCopy() # Get a copy of the board.
            boardCopy.makeMove(move) # Make the move.
            value = self.minimaxAlgo(depth - 1, boardCopy, -10000, 10000, not PlayerWhite) 

            # If new best move is found, store within the bestPossibleMoveFound variable.
            if team == "W" and value >= bestPossibleMove:
                bestPossibleMove = value
                bestPossibleMoveFound = move
            
            if team == "B" and value <= bestPossibleMove:
                bestPossibleMove = value
                bestPossibleMoveFound = move
        
        return bestPossibleMoveFound 

        # The minimaxAlgo algorithm is a recursive algorithm which determines the best possible outcome,
          #  with the assumption that one agent will always attempt to maximize the reward and one agent will
           # always attempt to minimize the reward
    def minimaxAlgo(self, depth, board, alpha, beta, PlayerWhite) -> int:
        
        # Return evaluation upon reaching leaf nodes.
        if depth == 0:
            return board.getEvaluation()
        
        # The maximizing player will always be the white team.
        if PlayerWhite==True:
            team = "W"
            bestPossibleMove = -999999
        else:
            team = "B"
            bestPossibleMove = 999999

        moves = board.getAllPossibleMoves(team)
        
        # Determine if player is attempting to maximize or minimize the board evaluation.
        if PlayerWhite==True:
            for move in moves:
                bestPossibleMove = max(bestPossibleMove, self.minimaxAlgo(depth - 1, board, alpha, beta, not PlayerWhite))
                alpha = max(alpha, bestPossibleMove)

                if alpha >= beta:
                    return bestPossibleMove
        else:            
            for move in moves:
                bestPossibleMove = min(bestPossibleMove, self.minimaxAlgo(depth - 1, board, alpha, beta, not PlayerWhite))
                beta = min(beta, bestPossibleMove)
                
                if alpha >= beta:
                    return bestPossibleMove
        
        return bestPossibleMove

def checkForGameEnd():

    if board.isStalemate("W") or board.isStalemate("B"):
        print("The game has ended in a draw.")
        return
    
    if board.isCheckmate("B"):
        print("The game has ended. {} has won.".format("White"))
        return

    if board.isCheckmate("W"):
        print("The game has ended. {} has won.".format("Black"))
        return

if __name__ == "__main__":

    depth = 4

    team = input("What team would you like to play as? Enter 'W' for white and 'B' for black: ")

    while team not in ['W', 'B']:
        team = input("What team would you like to play as? Enter 'W' for white and 'B' for black: ")

    if team == "W":
        team1 = "B"
        aiTeam = "W"
    else:
        team1="W"
        aiTeam = "B"
        
    #print("Your  Team is : ",team,"\n")
    #print("AI team is : ",aiTeam,"\n")

    board = Board()
    board.fillBoard()

    player1 = Player(team1, board, False)
    player2 = Player(aiTeam, board, True)

    if team == "W":
        startingPlayer = player1
        otherPlayer = player2
    else:
        startingPlayer = player2
        otherPlayer = player1

    #print("Starting Team is : ",startingPlayer,"\n")
    #print("AI team is : ",aiTeam,"\n")
        
    board.declareAsAI(aiTeam)

    board.printBoard()

    while True:
        command = input(
        "The following are the list of commands: \n"
        "move: Allows user to make a move.\n"
        "draw: Ends the game in a draw if 50 moves have been made without a pawn capture or move.\n")

        while command not in ["draw", "move"]:
            command = input(
                "Type a command: ")

        if command == "draw":

            if board.canDeclareDraw() == True:
                print("The game has ended in a draw.")
                break
            else:
                print("The game not be declared as a draw currently. Current count without a pawn move or capture: {}. Make a move.".format(
                    board.counter))

        else:

            checkForGameEnd()

            if startingPlayer.isAI == False:
                print("****** YOUR TURN ******")
                move = input("Type in a move in the format x1y1x2y2 (move: d2d4): ")
                while move not in startingPlayer.getAllPossibleMoves():
                    print("{} is an invalid move. Try again.".format(move))
                    move = input("Type in a move in the format x1y1x2y2 (move: d2d4): ")
                        
                startingPlayer.move(move)
                board.printBoard()
                checkForGameEnd()

                print("****** AI's TURN ******")
                otherPlayer.move(otherPlayer.getBestPossibleMove(depth))
            else:
                
                print("****** AI's TURN ******")
                startingPlayer.move(startingPlayer.getBestPossibleMove(depth))
                board.printBoard()
                checkForGameEnd()

                print("****** YOUR TURN ******")
                move = input("Type in a move in the format x1y1x2y2 (move: d2d4): ")
                while move not in otherPlayer.getAllPossibleMoves():
                    print("{} is an invalid move. Try again.".format(move))
                    move = input("Type in a move in the format x1y1x2y2 (move: d2d4): ")

                otherPlayer.move(move)
                
            board.printBoard()


KeyboardInterrupt: Interrupted by user